# Web spam detection through link-based features

Project for Information Retrieval exam at University of Trieste, January 2021

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import spam_detection as sd #python module containing custom functions
np.random.seed(2) #random seed for reproducibility

Web spam detection is a crucial issue for web search engines. In fact, ranking algorithms such as PageRank cannot explicitly penalize spam websites in favor of trustworthy ones, meaning that users may find very high-rank pages that have no useful content and are highly ranked because they are part of a link farm, a popular way to fool ranking algorithms.

###  The dataset
The [WEBSPAM-UK2006](https://chato.cl/webspam/datasets/uk2006/) dataset contains 11402 hosts in the `.uk` domain, of which 7866 are labeled as *spam* or *normal*. Newer datasets have been released by the same authors, but this 2006 version remains the one with the highest number of manually labeled samples.

The file `new_hostnames.csv` contains the names of the hosts in the dataset, while `webspam-uk2006-labels.txt` assigns to 8045 host names a label chosen among *spam*, *normal* or *undecided*. For the purposes of this project, *undecided*-labeled hosts were considered unlabeled, leaving only 7866 hosts labeled as *spam* or *normal*.

Finally, the file `uk-2006-05.hostgraph_weighted.txt` contains the weighted graph of the hosts, each row containing a host index, the indices of outlinked hosts and, for each of them, the number of outlinks.

The function `read_graph` returns a sparse `csr_matrix` $R$, with $R[i,j]$ equal to $0$ if there is no edge connecting host $i$ to host $j$ or to $\frac{1}{O[i]}$ where $O[i]$ is the total number of hosts outlinked by $i$.

Since PageRank algorithm requires a stochastic matrix, but there is no guarantee that each host has at least one outlink (dangling node problem), as proposed in [[1]](#references), an artificial node with a single self-loop was added to the graph, with ingoing edges from all dangling nodes.

In [2]:
hostnames=sd.hostnames_list('data/new_hostnames.csv')
labels_dict=sd.labels_dictionary('data/webspam-uk2006/webspam-uk2006-labels.txt')
labels, labeled_dataset=sd.make_dataset(labels_dict,hostnames)
R=sd.read_graph('data/uk-2006-05.hostgraph_weighted.txt',len(hostnames))

### PageRank

In the following cell PageRank is computed iteratively, according to the equation:

$$
{rank}_{k+1}=\frac{\alpha}{N}\mathbf{1}+(1-\alpha)R^T\cdot {rank}_k
$$

where ${rank}_k$ is the column vector storing the PageRank scores at step $k$, $\mathbf{1}$ is a column vector of ones, $N$ is the number of nodes (in this case 11403) and $\alpha$ is the teleporting factor. The iterative computation is performed up to a fixed precision of $\epsilon$, i.e. until $|{rank}_k-{rank}_{k-1}|_1 < \epsilon$.


In [3]:
alpha=.01
eps=1e-8

print("PageRank computation")
rank=sd.compute_PR(alpha,eps,R)

PageRank computation
PageRank computed


### An approximation for Personalized PageRank

Personalized PageRank is an algorithm directly derived from PageRank, whose result is a $N \times N$ matrix $PRM$ such that $PRM[i,j]$ is the contribution of node $i$ to the PageRank of node $j$. This implies that the sum over the columns of $PRM$ is equal to the PageRank vector.

Given a node index $v$, its corresponding contribution vector $cpr[v]$ is defined to be the row vector whose transpose is the $v$-th column of matrix $PRM$. This vector stores the contribution of all nodes to the PageRank of node $v$ and it will prove to be of particular interest when it comes to web spam detection.

However, Personalized PageRank computation is infeasible on large datasets, since it requires an iterative computation (conceptually identical to the one of PageRank) that includes, at each step, a matrix multiplication between two $N \times N$ matrices, one of which is non-sparse.

To address this problem, the authors of [[2]](#references) propose a local algorithm for the computation of $\delta$-approximations of contribution vectors.

Given a node $v$ and its contribution vector $c:=cpr[v]$, a $\delta$-approximation of $cpr(v)$ is a non-negative vector $c^*$ such that $c[u]-\delta rank[v] \leq c^*[u] \leq c[u]$ for all nodes $u$.

Since the following computations access repeatedly to the columns of matrix $R$, for performance reasons it is useful to store them in a list.

In [4]:
columns=sd.columns_list(R)

In [5]:
delta=1e-3

nl=len(labeled_dataset)
ap=np.zeros((nl,len(rank)))
print("Approximation of Personalized PageRank for labeled hosts")
for v in range(nl):
    print(str(v+1)+'/'+str(nl),end='\r')
    ap[v]=(sd.approximate_contributions(labeled_dataset[v], alpha, delta*rank[labeled_dataset[v]], rank[labeled_dataset[v]], columns))

Approximation of Personalized PageRank for labeled hosts


### Features for link-based web spam detection
Spam detection is particularly relevant for high PageRank hosts, since people tend to click on highly ranked pages, almost always within the first page of search engine results [[3]](#references).

However, we can see that PageRank alone is not able to filter out spam pages. In fact, if we restrict our view to the highest ranked 25% of the dataset, 144 hosts out of 2276 (total labeled hosts in this 25%) are labeled as spam, as opposed to 773/7866 on the entire dataset. The proportion of spam hosts drops from 9.8% to 6.3%, but that is surely not enough to consider PageRank as a spam detection or spam-robust algorithm.

In [6]:
n=25
labeled_top=sd.top_n_percent(n,rank,labeled_dataset)

y=labels[labeled_dataset]
y_top=y[labeled_top]

print("Total spam hosts: ", sum(y))
print("Total labeled hosts: ",len(y))
print("Total spam hosts %: ", 100*sum(y)/len(y),'\n')
print("Spam hosts in top 25%: ", sum(y_top))
print("Labeled hosts in top 25%: ", len(y_top))
print("Spam hosts % in top 25%: ", 100*sum(y_top)/len(y_top))

Total spam hosts:  773
Total labeled hosts:  7866
Total spam hosts %:  9.827103991863718 

Spam hosts in top 25%:  144
Labeled hosts in top 25%:  2276
Spam hosts % in top 25%:  6.3268892794376095


There are two basic approaches to web spam detection: content-based and link-based.

In a content-based setting the `HTML` code of pages is scanned and some features are computed, usually related to text (for example, average word length or fraction of visible text). On the other hand, in a link-based setting (the one explored in this notebook), information is obtained exclusively from the web graph and computations performed over it (such as ranking algorithms).

Some very trivial link-based features that can be computed even before performing PageRank are indegree and outdegree, defined as follows:
 - Indegree: total number of incoming links to a host
 - Outdegree: total number of outgoing links from a host

Other useful features can be computed from the contribution vector of a node (or from a $\delta$-approximation):
 - Size of $\delta$-significant contributing set: for a node $v$, this feature is defined as $cs\_size=|S_{\delta}|=|\{u|c^*(u)>\delta rank(v)\}|$.
 - Contribution from vertices in the $\delta$-significant contributing set: $cs\_contribution=\sum_{u \in S_{\delta}} \frac{c^*(u)}{rank(v)}$ 
 - $l_2$ norm of $\delta$-significant contributing vector: $l_2\_norm=\sum_{u \in S_{\delta}} (\frac{c^*(u)}{rank(v)})^2$

These features, together with PageRank scores, can be used to train a binary classifier for spam detection.

In [7]:
x=np.zeros((nl,6))
indegree,outdegree,cs_size,cs_contribution,l2_norm=sd.extract_features(R,delta,ap,labeled_dataset,rank)
x[:,0]=rank[labeled_dataset]
x[:,1]=indegree
x[:,2]=outdegree
x[:,3]=cs_size
x[:,4]=cs_contribution
x[:,5]=l2_norm

### Evaluation of classifiers

Some Machine Learning can be used to detect spam websites. Three different approaches are compared: Logistic Regression, Decision Tree and Random Forest. For all these classifiers, the implementation from `scikit-learn` library is used with its default hyperparameter values, with the exception of the parameter `class_weight`, which is set to `balanced` to try to mitigate the unbalance in the dataset between *spam* and *normal* labeled data points. 

The fit and test of these models is performed through a 10-fold cross validation. Regarding performance metrics, in a classification setting accuracy is obviously to be taken into account but, in this case, not sufficient to compare the models. In fact, given how unbalanced the dataset is, even a naive classifier that classifies all hosts as *normal* would achieve over 90% accuracy despite being completely useless for spam detection. To solve this problem, class-specific precision and recall statistics are taken into account for *spam* class. In particular, since spam labeling might be safety critical, one could imagine recall to be particularly relevant.

Tree-based classifiers work definitely better than Logistic Regression, scoring accuracies over 90% while attaining reasonable values for precision and recall on *spam* samples. However, Logistic Regression scores an outstading 95% recall on *spam* meaning that it succeeds in retrieving almost all spam hosts (at the cost of many false alarms, captured by low precision score).

In [8]:
k=10

print("Logistic Regression")
clf=LogisticRegression(class_weight='balanced')
sd.print_prediction_metrics(clf,x,y,k)

print("\nDecision Tree")
clf=DecisionTreeClassifier(class_weight='balanced')
sd.print_prediction_metrics(clf,x,y,k)

print("\nRandom Forest")
clf=RandomForestClassifier(class_weight='balanced_subsample')
sd.print_prediction_metrics(clf,x,y,k)

Logistic Regression
Accuracy:  0.738367658276125
Precision on spam:  0.26644856415848783
Recall on spam:  0.9482535575679172

Decision Tree
Accuracy:  0.9188914314772438
Precision on spam:  0.5896414342629482
Recall on spam:  0.574385510996119

Random Forest
Accuracy:  0.9359267734553776
Precision on spam:  0.71869918699187
Recall on spam:  0.5717981888745148


Once again, it is particularly interesting to restrict the view to the top 25% highest ranked hosts, since these are the ones that users are most likely to run into.

Here there is no huge gap in accuracy between the three classifiers, with values well over 95% for all of them. Tree-based methods confirm their higher precision on *spam* samples but, because of the different misclassification cost between the two class and because of a (almost) perfect recall score on *spam* Logistic Regression would probably be preferrable.

In [9]:
x_top=x[labeled_top]

print("Logistic Regression")
clf=LogisticRegression(class_weight='balanced')
sd.print_prediction_metrics(clf,x_top,y_top,k)

print("\nDecision Tree")
clf=DecisionTreeClassifier(class_weight='balanced')
sd.print_prediction_metrics(clf,x_top,y_top,k)

print("\nRandom Forest")
clf=RandomForestClassifier(class_weight='balanced_subsample')
sd.print_prediction_metrics(clf,x_top,y_top,k)

Logistic Regression
Accuracy:  0.9556239015817223
Precision on spam:  0.588477366255144
Recall on spam:  0.9930555555555556

Decision Tree
Accuracy:  0.9740773286467487
Precision on spam:  0.8014184397163121
Recall on spam:  0.7847222222222222

Random Forest
Accuracy:  0.9793497363796133
Precision on spam:  0.8646616541353384
Recall on spam:  0.7986111111111112


### References
<a id='references'></a>
- [1] R. Andersen, C. Borgs, J. Chayes, J. Hopcroft, K. Jain, V. Mirrokni and S. Teng. [Robust PageRank and locally computable spam detection features](https://dl.acm.org/doi/10.1145/1451983.1452000). In Proceedings of the 4th international workshop on Adversarial information retrieval on the web (AIRWeb '08), 2008.
- [2] R. Andersen, C. Borgs, J. Chayes, J. Hopcroft, V. Mirrokni and S. Teng. [Local computation of PageRank contributions](https://link.springer.com/chapter/10.1007/978-3-540-77004-6_12). In 5th International Workshop of Algorithms and Models for the Web-Graph, 2007.
- [3] C. Barry, M. Lardner. [A Study of First Click Behaviour and User Interaction on the Google SERP](https://link.springer.com/chapter/10.1007/978-1-4419-9790-6_7). In: Pokorny J. et al. (eds) Information Systems Development. Springer, New York, 2011.